In [12]:
import os
os.chdir('/home/benjamindougl_snhu/Desktop')
print("Current directory:", os.getcwd())
from jupyter_dash import JupyterDash
import dash_leaflet as dl
from dash import dcc, html, dash_table
from dash.dependencies import Input, Output, State
import plotly.express as px
import pandas as pd
import base64
from animal_shelter import AnimalShelter

# Initialize the database connection
username = "aacuser"
password = "SNHU1234"

# Create the Dash app
app = JupyterDash(__name__)

# Load and encode the logo
image_filename = 'grazioso_salvare_logo.png'

# Define the rescue type criteria
RESCUE_TYPES = {
    'water': {
        'query': {
            'breed': {'$in': ['Labrador Retriever Mix', 'Chesapeake Bay Retriever', 'Newfoundland']},
            'sex': 'Female',
            'age_upon_outcome_in_weeks': {'$gte': 26, '$lte': 156}
        }
    },
    'mountain': {
        'query': {
            'breed': {'$in': ['German Shepherd', 'Alaskan Malamute', 'Old English Sheepdog', 
                             'Siberian Husky', 'Rottweiler']},
            'sex': 'Male',
            'age_upon_outcome_in_weeks': {'$gte': 26, '$lte': 156}
        }
    },
    'disaster': {
        'query': {
            'breed': {'$in': ['Doberman Pinscher', 'German Shepherd', 'Golden Retriever',
                             'Bloodhound', 'Rottweiler']},
            'sex': 'Male',
            'age_upon_outcome_in_weeks': {'$gte': 20, '$lte': 300}
        }
    }
}

# Define the layout
app.layout = html.Div([
    # Header with logo and creator info
    html.Div([
        html.A([
            html.Img(
        
                style={'height': '100px'}
            )
        ], href='https://www.snhu.edu'),
        html.Div('Created by: Ben Douglas', style={'text-align': 'right'})
    ], style={'display': 'flex', 'justify-content': 'space-between', 'align-items': 'left'}),
    
    # Filter options
    html.Div([
        dcc.RadioItems(
            id='filter-type',
            options=[
                {'label': 'Water Rescue', 'value': 'water'},
                {'label': 'Mountain/Wilderness Rescue', 'value': 'mountain'},
                {'label': 'Disaster/Individual Tracking', 'value': 'disaster'},
                {'label': 'Reset', 'value': 'reset'}
            ],
            value='reset'
        )
    ]),
    
    # Data table
    dash_table.DataTable(
        id='datatable-id',
        page_current=0,
        page_size=10,
        page_action='native',
        sort_action='native',
        sort_mode='multi',
        selected_rows=[],
        style_data_conditional=[{
            'if': {'row_index': 'odd'},
            'backgroundColor': 'rgb(248, 248, 248)'
        }]
    ),
    
    # Charts container
    html.Div([
        # Pie chart
        html.Div([
            dcc.Graph(id='pie-chart')
        ], className='six columns'),
        
        # Map
        html.Div([
            dl.Map(id='map', style={'height': '500px'})
        ], className='six columns')
    ], className='row')
])

# Callback for filtering data
@app.callback(
    [Output('datatable-id', 'data'),
     Output('datatable-id', 'columns')],
    [Input('filter-type', 'value')]
)
def update_dashboard(filter_type):
    if filter_type == 'reset':
        df = pd.DataFrame.from_records(db.read({}))
    else:
        df = pd.DataFrame.from_records(db.read(RESCUE_TYPES[filter_type]['query']))
    
    df.drop(columns=['_id'], inplace=True)
    columns = [{"name": i, "id": i} for i in df.columns]
    
    return df.to_dict('records'), columns

# Callback for pie chart
@app.callback(
    Output('pie-chart', 'figure'),
    [Input('datatable-id', 'data')]
)
def update_pie_chart(data):
    if not data:
        return {}
    
    df = pd.DataFrame(data)
    breed_counts = df['breed'].value_counts()
    
    return px.pie(
        values=breed_counts.values,
        names=breed_counts.index,
        title='Dog Breeds Distribution'
    )

# Callback for map
@app.callback(
    Output('map', 'children'),
    [Input('datatable-id', 'data'),
     Input('datatable-id', 'selected_rows')]
)
def update_map(data, selected_rows):
    if not data or not selected_rows:
        return dl.TileLayer()
    
    df = pd.DataFrame(data)
    row = df.iloc[selected_rows[0]]
    
    return [
        dl.TileLayer(),
        dl.Marker(
            position=[row['location_lat'], row['location_long']],
            children=[
                dl.Tooltip(row['breed']),
                dl.Popup([
                    html.H3("Animal Details"),
                    html.P(f"Name: {row['name']}"),
                    html.P(f"Breed: {row['breed']}"),
                    html.P(f"Age: {row['age_upon_outcome']}")
                ])
            ]
        )
    ]

if __name__ == '__main__':
    app.run_server(debug=True)

Current directory: /home/benjamindougl_snhu/Desktop
Dash app running on http://127.0.0.1:27232/
